In [2]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
openai.log = "none"

In [3]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne

In [ ]:
! pip install nltk 

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
import openai
openai.api_key = "sk-xxx"

In [5]:
import tiktoken
from llama_index import ServiceContext
from llama_index.callbacks import CallbackManager, TokenCountingHandler

token_counter = TokenCountingHandler(
    tokenizer = tiktoken.encoding_for_model("text-embedding-ada-002").encode,
    verbose = True,
)
callback_manager = CallbackManager([token_counter])

service_context = ServiceContext.from_defaults(callback_manager=callback_manager)

In [7]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage

try:
    storage_context = StorageContext.from_defaults(persist_dir='./storage/cache/andrew/sleep')
    index = load_index_from_storage(storage_context)
    print('loading from disk')
except:
    documents = SimpleDirectoryReader('assets/AndrewHuberman/sleep').load_data()
    index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    index.storage_context.persist(persist_dir='./storage/cache/andrew/sleep/')
    print('persisting to disk')

print(token_counter.total_embedding_token_count)

Embedding Token Usage: 9760
Embedding Token Usage: 9767
Embedding Token Usage: 9448
Embedding Token Usage: 9574
Embedding Token Usage: 9608
Embedding Token Usage: 9307
Embedding Token Usage: 8760
Embedding Token Usage: 9692
Embedding Token Usage: 9648
Embedding Token Usage: 9512
Embedding Token Usage: 9136
Embedding Token Usage: 9807
Embedding Token Usage: 9540
Embedding Token Usage: 1397
persisting to disk
124956


In [14]:
token_counter.reset_counts()

In [15]:
response = index.as_query_engine().query("How does sleep enhance learning memory?")
print('Embedding tokens: ', token_counter.total_embedding_token_count, '\n',
    'LLM prompts: ', token_counter.prompt_llm_token_count, '\n',
    'LLM completitions: ', token_counter.completion_llm_token_count, '\n',
    'Total LLM token count: ', token_counter.total_llm_token_count, '\n',
)
print(response)

Embedding Token Usage: 7
LLM Prompt Token Usage: 2060
LLM Completion Token Usage: 229
Embedding tokens:  7 
 LLM prompts:  2060 
 LLM completitions:  229 
 Total LLM token count:  2289 

Sleep enhances learning and memory by playing a crucial role in the consolidation and processing of information. During slow wave sleep, which primarily occurs in the early part of the night, motor learning and the learning of specific details about events take place. This type of sleep is important for motor skill learning and the formation of detailed memories. Slow wave sleep is characterized by the absence of acetylcholine and the presence of large amplitude brain activity. It is also associated with the release of neuromodulators such as norepinephrine and serotonin. On the other hand, rapid eye movement (REM) sleep, which occurs throughout the night with a higher percentage towards morning, is involved in memory consolidation. Dreams during REM sleep often incorporate new sensory motor experience

In [ ]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor, KeywordNodePostprocessor
from llama_index.response_synthesizers import get_response_synthesizer

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=4,
)
s_processor = SimilarityPostprocessor(similarity_cutoff=0.83)
k_processor = KeywordNodePostprocessor(
    exclude_keywords=["supplements"]
)

response_synthesizer = get_response_synthesizer(
    response_mode = "compact"
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[k_processor],
    response_synthesizer=response_synthesizer
)


In [ ]:

response = query_engine.query("What can I do to sleep better?")

print(response.source_nodes)